# [PMC6620049](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6620049/)


|sp|tissue|status|
|:-:|:-:|:-:|
|h|blood|-|
|h|liver|tumor|
|m|liver|tumor|
|m|liver|healthy|


+ immue cell 大类群
+ immue cell 亚类
    + Neutrophil
    + Dendritic Cells
    + Monocyte

In [1]:
from json import dumps, loads
from pathlib import Path

import numpy as np
import pandas as pd
import scanpy as sc
from IPython.display import display
from scipy.sparse import csr_matrix

p_link = Path("/public/workspace/licanchengup/link")
p_disease = p_link.joinpath("disease")
p_disease_info = p_disease.joinpath("GEO-DataSet_info.json")
if p_disease_info.exists():
    info = loads(p_disease_info.read_text())
else:
    info = []
display(len(info))

p_root = p_disease.joinpath("dataset", Path(".").absolute().name)
print("cd {}".format(p_root.absolute()))
print("{}_RAW".format(p_root.absolute().name))

24

cd /public/workspace/licanchengup/link/disease/dataset/LungCancer_GSE127465
LungCancer_GSE127465_RAW


In [2]:
# df_meta_h = pd.read_csv("GSE127465_human_cell_metadata_54773x25.tsv.gz", sep="\t")
# df_meta_m = pd.read_csv("GSE127465_mouse_cell_metadata_15939x12.tsv.gz", sep="\t")

# print(*df_meta_h.columns, sep="\n")

# display(df_meta_h.head(2), df_meta_h.shape)
# display(df_meta_m.head(2), df_meta_m.shape)

> ruru_97

后缀 t 和 h 指代 Tumor or healthy

但是human_h中的Tissue是blood哎

In [3]:
print(*[i.name for i in Path("ruru_97_input").iterdir() if i.is_file()], sep="\n")

human_t.h5ad
view.ipynb
mouse_h.h5ad
mouse_t.h5ad
human_h.h5ad


In [4]:
if not Path("human_t_ann.csv").exists():
    adata_h_t = sc.read_h5ad("ruru_97_input/human_t.h5ad")
    adata_h_t.obs.to_csv("human_t_ann.csv", index=True)
    temp = (adata_h_t.X[adata_h_t.X > 0]).copy()
    display(temp[:10])
if not Path("mouse_t_ann.csv").exists():
    adata_m_t = sc.read_h5ad("ruru_97_input/mouse_t.h5ad")
    adata_m_t.obs.to_csv("mouse_t_ann.csv", index=True)
    temp = (adata_m_t.X[adata_m_t.X > 0]).copy()
    display(temp[:10])

/public/workspace/licanchengup/apps/miniconda3/envs/env_came/lib/python3.8/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


array([2., 1., 1., 2., 1., 1., 1., 4., 1., 3.], dtype=float32)

/public/workspace/licanchengup/apps/miniconda3/envs/env_came/lib/python3.8/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


array([1., 1., 1., 2., 1., 1., 1., 2., 1., 3.], dtype=float32)

# manipulate the obs

In [5]:
df_obs_h = pd.read_csv("human_t_ann.csv", index_col=0)
df_obs_m = pd.read_csv("mouse_t_ann.csv", index_col=0)
display(df_obs_h.head(2), df_obs_h.shape, df_obs_m.head(2), df_obs_m.shape)

,Patient,Tissue,Barcoding emulsion,Library,Barcode,Total counts,Percent counts from mitochondrial genes,Most likely LM22 cell type,cell_type,Minor subset,...,used_in_NSCLC_immune,x_NSCLC_immune,y_NSCLC_immune,used_in_NSCLC_non_immune,x_NSCLC_non_immune,y_NSCLC_non_immune,used_in_blood,x_blood,y_blood,newbarcode
0,p1,tumor,p1t,p1t1,bcIIOD,18765.0,5.371703,Macrophages M0,Patient1-specific,Pt1D_MIAT,...,False,NaN,NaN,True,1449.350816,-341.447361,False,NaN,NaN,p1t1bcIIOD
1,p1,tumor,p1t,p1t1,bcHTNA,7516.0,3.579031,Macrophages M2,tMoMacDC,tMac2,...,True,1654.548012,-1145.346928,False,NaN,NaN,False,NaN,NaN,p1t1bcHTNA


(40362, 26)

,Tumor or healthy,Biological replicate,Library,Barcode,Library prep batch,Total counts,Percent counts from mitochondrial genes,Most likely Immgen cell type,cell_type,Minor subset,x,y,newbarcode
0,t,Tumor-bearing 1,t_1_1,bc0001,round1_20151128,7927.0,3.393,MF_Thio5_II+480int_PC,tMoMacDC,Mac1,1174.120533,-217.075559,t_1_1bc0001
1,t,Tumor-bearing 1,t_1_1,bc0002,round1_20151128,5665.0,6.461,DC_103-11b+24+_Lu,tMoMacDC,DC1,1068.808968,-37.966851,t_1_1bc0002


(9201, 13)

In [6]:
display(df_obs_h["newbarcode"].is_unique, df_obs_m["newbarcode"].is_unique)

True

True

In [7]:
df_obs = df_obs_h
display(df_obs.head(2), df_obs.shape)
temp = df_obs.apply(lambda row: "{Barcode};{Library}".format(**row), axis=1)
# make sure temp ( new_index ) is unique
assert temp.is_unique, "temp ( new_index ) is not unique"
df_obs["new_index"] = temp.to_numpy()
display(df_obs.head(2), df_obs.shape)
del temp, df_obs

,Patient,Tissue,Barcoding emulsion,Library,Barcode,Total counts,Percent counts from mitochondrial genes,Most likely LM22 cell type,cell_type,Minor subset,...,used_in_NSCLC_immune,x_NSCLC_immune,y_NSCLC_immune,used_in_NSCLC_non_immune,x_NSCLC_non_immune,y_NSCLC_non_immune,used_in_blood,x_blood,y_blood,newbarcode
0,p1,tumor,p1t,p1t1,bcIIOD,18765.0,5.371703,Macrophages M0,Patient1-specific,Pt1D_MIAT,...,False,NaN,NaN,True,1449.350816,-341.447361,False,NaN,NaN,p1t1bcIIOD
1,p1,tumor,p1t,p1t1,bcHTNA,7516.0,3.579031,Macrophages M2,tMoMacDC,tMac2,...,True,1654.548012,-1145.346928,False,NaN,NaN,False,NaN,NaN,p1t1bcHTNA


(40362, 26)

,Patient,Tissue,Barcoding emulsion,Library,Barcode,Total counts,Percent counts from mitochondrial genes,Most likely LM22 cell type,cell_type,Minor subset,...,x_NSCLC_immune,y_NSCLC_immune,used_in_NSCLC_non_immune,x_NSCLC_non_immune,y_NSCLC_non_immune,used_in_blood,x_blood,y_blood,newbarcode,new_index
0,p1,tumor,p1t,p1t1,bcIIOD,18765.0,5.371703,Macrophages M0,Patient1-specific,Pt1D_MIAT,...,NaN,NaN,True,1449.350816,-341.447361,False,NaN,NaN,p1t1bcIIOD,bcIIOD;p1t1
1,p1,tumor,p1t,p1t1,bcHTNA,7516.0,3.579031,Macrophages M2,tMoMacDC,tMac2,...,1654.548012,-1145.346928,False,NaN,NaN,False,NaN,NaN,p1t1bcHTNA,bcHTNA;p1t1


(40362, 27)

In [8]:
df_obs = df_obs_m
display(df_obs.head(2), df_obs.shape)
temp = df_obs.apply(lambda row: "{Barcode};{Library}".format(**row), axis=1)
# make sure temp ( new_index ) is unique
assert temp.is_unique, "temp ( new_index ) is not unique"
df_obs["new_index"] = temp.to_numpy()
display(df_obs.head(2), df_obs.shape)
del temp, df_obs

,Tumor or healthy,Biological replicate,Library,Barcode,Library prep batch,Total counts,Percent counts from mitochondrial genes,Most likely Immgen cell type,cell_type,Minor subset,x,y,newbarcode
0,t,Tumor-bearing 1,t_1_1,bc0001,round1_20151128,7927.0,3.393,MF_Thio5_II+480int_PC,tMoMacDC,Mac1,1174.120533,-217.075559,t_1_1bc0001
1,t,Tumor-bearing 1,t_1_1,bc0002,round1_20151128,5665.0,6.461,DC_103-11b+24+_Lu,tMoMacDC,DC1,1068.808968,-37.966851,t_1_1bc0002


(9201, 13)

,Tumor or healthy,Biological replicate,Library,Barcode,Library prep batch,Total counts,Percent counts from mitochondrial genes,Most likely Immgen cell type,cell_type,Minor subset,x,y,newbarcode,new_index
0,t,Tumor-bearing 1,t_1_1,bc0001,round1_20151128,7927.0,3.393,MF_Thio5_II+480int_PC,tMoMacDC,Mac1,1174.120533,-217.075559,t_1_1bc0001,bc0001;t_1_1
1,t,Tumor-bearing 1,t_1_1,bc0002,round1_20151128,5665.0,6.461,DC_103-11b+24+_Lu,tMoMacDC,DC1,1068.808968,-37.966851,t_1_1bc0002,bc0002;t_1_1


(9201, 14)

In [9]:
display(df_obs_h.head(2), df_obs_h.shape)
display(df_obs_m.head(2), df_obs_m.shape)

,Patient,Tissue,Barcoding emulsion,Library,Barcode,Total counts,Percent counts from mitochondrial genes,Most likely LM22 cell type,cell_type,Minor subset,...,x_NSCLC_immune,y_NSCLC_immune,used_in_NSCLC_non_immune,x_NSCLC_non_immune,y_NSCLC_non_immune,used_in_blood,x_blood,y_blood,newbarcode,new_index
0,p1,tumor,p1t,p1t1,bcIIOD,18765.0,5.371703,Macrophages M0,Patient1-specific,Pt1D_MIAT,...,NaN,NaN,True,1449.350816,-341.447361,False,NaN,NaN,p1t1bcIIOD,bcIIOD;p1t1
1,p1,tumor,p1t,p1t1,bcHTNA,7516.0,3.579031,Macrophages M2,tMoMacDC,tMac2,...,1654.548012,-1145.346928,False,NaN,NaN,False,NaN,NaN,p1t1bcHTNA,bcHTNA;p1t1


(40362, 27)

,Tumor or healthy,Biological replicate,Library,Barcode,Library prep batch,Total counts,Percent counts from mitochondrial genes,Most likely Immgen cell type,cell_type,Minor subset,x,y,newbarcode,new_index
0,t,Tumor-bearing 1,t_1_1,bc0001,round1_20151128,7927.0,3.393,MF_Thio5_II+480int_PC,tMoMacDC,Mac1,1174.120533,-217.075559,t_1_1bc0001,bc0001;t_1_1
1,t,Tumor-bearing 1,t_1_1,bc0002,round1_20151128,5665.0,6.461,DC_103-11b+24+_Lu,tMoMacDC,DC1,1068.808968,-37.966851,t_1_1bc0002,bc0002;t_1_1


(9201, 14)

## process cell_type  Minor subset

In [10]:
print(*df_obs_h.columns, sep="\n")

Patient
Tissue
Barcoding emulsion
Library
Barcode
Total counts
Percent counts from mitochondrial genes
Most likely LM22 cell type
cell_type
Minor subset
used_in_NSCLC_all_cells
x_NSCLC_all_cells
y_NSCLC_all_cells
used_in_NSCLC_and_blood_immune
x_NSCLC_and_blood_immune
y_NSCLC_and_blood_immune
used_in_NSCLC_immune
x_NSCLC_immune
y_NSCLC_immune
used_in_NSCLC_non_immune
x_NSCLC_non_immune
y_NSCLC_non_immune
used_in_blood
x_blood
y_blood
newbarcode
new_index


In [11]:
df_obs_h = df_obs_h[df_obs_h["used_in_NSCLC_immune"]]

In [12]:
# display(
#     df_obs_h["cell_type"].value_counts(sort=False),
#     df_obs_h["Minor subset"].value_counts(sort=False),
# )

df_obs_h = df_obs_h.assign(
    CellType=df_obs_h["cell_type"].str.replace("^t", "", regex=True),
    SubCellType=df_obs_h["Minor subset"].str.replace("^t", "", regex=True),
)


display(
    df_obs_h["CellType"].value_counts(sort=False),
    df_obs_h["SubCellType"].value_counts(sort=False),
)

CellType
MoMacDC          9372
Mast cells        732
T cells         12776
Neutrophils      2728
NK cells         1116
Plasma cells     2197
B cells          5218
pDC               311
RBC               108
Name: count, dtype: int64

SubCellType
Mac2        919
Mono1       698
Mast1       643
Mac6        284
MacCycl     200
T2         6654
T4          342
T1         2397
N5          902
T3         2163
NK1         886
MonoDC     1097
PC6         796
DC2         330
Mac7        412
Mac3       2554
Mac1        949
Mac4        418
Mac8        502
N3          425
Mono2       252
T6          691
N4          789
PC3         369
Mac9         94
PC1         638
B          5218
pDC         311
Mono3       226
DC3         101
N2           86
Mac5        165
PC4         244
T7          233
DC1         171
N1          526
T5          296
NK2         230
Mast2        89
PC2          77
PC5          73
RBC         108
Name: count, dtype: int64

In [13]:
# display(
#     df_obs_m["cell_type"].value_counts(sort=False),
#     df_obs_m["Minor subset"].value_counts(sort=False),
# )

df_obs_m = df_obs_m.assign(
    CellType=df_obs_m["cell_type"].str.replace("^t", "", regex=True),
    SubCellType=df_obs_m["Minor subset"].copy(),
)


display(
    df_obs_m["CellType"].value_counts(sort=False),
    df_obs_m["SubCellType"].value_counts(sort=False),
)

CellType
MoMacDC        1768
Neutrophils    3593
T cells        1491
pDC              52
NK cells        400
B cells        1874
Basophils        23
Name: count, dtype: int64

SubCellType
Mac1          149
DC1           119
DC3           293
Mono3         210
Mac2          100
N4           1782
T2            141
Mono1         407
T3             52
T1           1298
N1            968
N5            580
DC2           106
N6            109
Mac3           23
pDC            52
NK cells      400
MonoDC         76
B cells      1874
Mono2         154
N2             24
Mac4          131
Basophils      23
N3            130
Name: count, dtype: int64

## 尝试对齐

In [14]:
display(
    pd.concat(
        {
            "h": df_obs_h["CellType"].value_counts(),
            "m": df_obs_m["CellType"].value_counts(),
        },
        axis=1,
    ).sort_index()
)

# display(
#     pd.concat(
#         {
#             "h": df_obs_h["SubCellType"].value_counts(),
#             "m": df_obs_m["SubCellType"].value_counts(),
#         },
#         axis=1,
#     ).sort_index()
# )

,h,m
CellType,,
B cells,5218.0,1874.0
Basophils,NaN,23.0
Mast cells,732.0,NaN
MoMacDC,9372.0,1768.0
NK cells,1116.0,400.0
Neutrophils,2728.0,3593.0
Plasma cells,2197.0,NaN
RBC,108.0,NaN
T cells,12776.0,1491.0


In [15]:
display(
    pd.concat(
        {
            "h": df_obs_h.query("CellType ==  'Neutrophils'")[
                "SubCellType"
            ].value_counts(),
            "m": df_obs_m.query("CellType ==  'Neutrophils'")[
                "SubCellType"
            ].value_counts(),
        },
        axis=1,
    ).sort_index()
)

,h,m
SubCellType,,
N1,526.0,968
N2,86.0,24
N3,425.0,130
N4,789.0,1782
N5,902.0,580
N6,NaN,109


In [16]:
display(
    pd.concat(
        {
            "h": df_obs_h[df_obs_h["SubCellType"].str.match("^p?DC")][
                "SubCellType"
            ].value_counts(),
            "m": df_obs_m[df_obs_m["SubCellType"].str.match("^p?DC")][
                "SubCellType"
            ].value_counts(),
        },
        axis=1,
    ).sort_index()
)

,h,m
SubCellType,,
DC1,171,119
DC2,330,106
DC3,101,293
pDC,311,52


In [17]:
display(
    pd.concat(
        {
            "h": df_obs_h[df_obs_h["SubCellType"].str.startswith("Mono")][
                "SubCellType"
            ].value_counts(),
            "m": df_obs_m[df_obs_m["SubCellType"].str.startswith("Mono")][
                "SubCellType"
            ].value_counts(),
        },
        axis=1,
    ).sort_index()
)

,h,m
SubCellType,,
Mono1,698,407
Mono2,252,154
Mono3,226,210
MonoDC,1097,76


In [18]:
df_obs_h.to_csv("human_t_ann.csv", index=True)
df_obs_m.to_csv("mouse_t_ann.csv", index=True)
del df_obs_h,df_obs_m

# manipulate the adata

In [19]:
out = "human_t.h5ad"
def manipulate_adata(out):
    adata = sc.read_h5ad("ruru_97_input/{}".format(out))
    # display(adata,adata.obs.head(2))
    df_obs = pd.read_csv(out.replace(".h5ad","_ann.csv"),index_col=0)
    display(df_obs.head(2),df_obs.shape)
    df_obs.index = df_obs.index.astype(str)
    adata = adata[adata.obs.index.isin(df_obs.index)]
    assert (adata.obs["newbarcode"].to_numpy() == df_obs["newbarcode"].to_numpy()).all(),"[Error] 截取错误"
    display(adata,adata.obs.head(2))
    adata.obs = adata.obs.loc[:,[]].join(df_obs)
    adata.obs.index = adata.obs["new_index"].to_numpy()
    adata.obs = adata.obs.drop(columns=["new_index"])
    adata.obs.to_csv(out.replace(".h5ad","_ann.csv"),index=True)
    display(type(adata.X))
    adata.X = csr_matrix(adata.X)
    display(type(adata.X))
    adata.write_h5ad(out)
    print("[out] {}".format(out))

manipulate_adata("human_t.h5ad")
manipulate_adata("mouse_t.h5ad")

/public/workspace/licanchengup/apps/miniconda3/envs/env_came/lib/python3.8/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


,Patient,Tissue,Barcoding emulsion,Library,Barcode,Total counts,Percent counts from mitochondrial genes,Most likely LM22 cell type,cell_type,Minor subset,...,used_in_NSCLC_non_immune,x_NSCLC_non_immune,y_NSCLC_non_immune,used_in_blood,x_blood,y_blood,newbarcode,new_index,CellType,SubCellType
1,p1,tumor,p1t,p1t1,bcHTNA,7516.0,3.579031,Macrophages M2,tMoMacDC,tMac2,...,False,NaN,NaN,False,NaN,NaN,p1t1bcHTNA,bcHTNA;p1t1,MoMacDC,Mac2
3,p1,tumor,p1t,p1t1,bcHNVA,5684.0,4.574243,Macrophages M2,tMoMacDC,tMac2,...,False,NaN,NaN,False,NaN,NaN,p1t1bcHNVA,bcHNVA;p1t1,MoMacDC,Mac2


(34558, 29)

View of AnnData object with n_obs × n_vars = 34558 × 41861
    obs: 'Patient', 'Tissue', 'Barcoding emulsion', 'Library', 'Barcode', 'Total counts', 'Percent counts from mitochondrial genes', 'Most likely LM22 cell type', 'cell_type', 'Minor subset', 'used_in_NSCLC_all_cells', 'x_NSCLC_all_cells', 'y_NSCLC_all_cells', 'used_in_NSCLC_and_blood_immune', 'x_NSCLC_and_blood_immune', 'y_NSCLC_and_blood_immune', 'used_in_NSCLC_immune', 'x_NSCLC_immune', 'y_NSCLC_immune', 'used_in_NSCLC_non_immune', 'x_NSCLC_non_immune', 'y_NSCLC_non_immune', 'used_in_blood', 'x_blood', 'y_blood', 'newbarcode'

,Patient,Tissue,Barcoding emulsion,Library,Barcode,Total counts,Percent counts from mitochondrial genes,Most likely LM22 cell type,cell_type,Minor subset,...,used_in_NSCLC_immune,x_NSCLC_immune,y_NSCLC_immune,used_in_NSCLC_non_immune,x_NSCLC_non_immune,y_NSCLC_non_immune,used_in_blood,x_blood,y_blood,newbarcode
1,p1,tumor,p1t,p1t1,bcHTNA,7516.0,3.579031,Macrophages M2,tMoMacDC,tMac2,...,True,1654.548012,-1145.346928,False,NaN,NaN,False,NaN,NaN,p1t1bcHTNA
3,p1,tumor,p1t,p1t1,bcHNVA,5684.0,4.574243,Macrophages M2,tMoMacDC,tMac2,...,True,1651.629433,-1211.471793,False,NaN,NaN,False,NaN,NaN,p1t1bcHNVA


...

View of AnnData object with n_obs × n_vars = 9201 × 28205
    obs: 'Tumor or healthy', 'Biological replicate', 'Library', 'Barcode', 'Library prep batch', 'Total counts', 'Percent counts from mitochondrial genes', 'Most likely Immgen cell type', 'cell_type', 'Minor subset', 'x', 'y', 'newbarcode'

,Tumor or healthy,Biological replicate,Library,Barcode,Library prep batch,Total counts,Percent counts from mitochondrial genes,Most likely Immgen cell type,cell_type,Minor subset,x,y,newbarcode
0,t,Tumor-bearing 1,t_1_1,bc0001,round1_20151128,7927.0,3.393,MF_Thio5_II+480int_PC,tMoMacDC,Mac1,1174.120533,-217.075559,t_1_1bc0001
1,t,Tumor-bearing 1,t_1_1,bc0002,round1_20151128,5665.0,6.461,DC_103-11b+24+_Lu,tMoMacDC,DC1,1068.808968,-37.966851,t_1_1bc0002


numpy.ndarray

scipy.sparse._csr.csr_matrix

[out] mouse_t.h5ad


In [20]:
print("\n[finish]\n".center(100,"-"))

---------------------------------------------
[finish]
---------------------------------------------
